In [54]:
import pandas as pd
import re
import os
import json

# Cleaning and tokenization logic
def clean_text(text):
    text = str(text).strip().lower()
    text = re.sub(r'\s+', ' ', text)  # normalize whitespace
    text = re.sub(r'[^\w\s.%]', '', text)  # remove punctuation except % and .
    return text

def tokenize_text(text):
    return re.findall(r'\b\w+\b', text)

# Step 1: Load and preprocess FAQ
def load_and_preprocess_faq(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        raw = json.load(f)

    all_entries = []
    for category in raw["categories"]:
        for q in category["questions"]:
            q_clean = clean_text(q["question"])
            a_clean = clean_text(q["answer"])
            all_entries.append({
                "category": category["category"],
                "question": q_clean,
                "question_tokens": tokenize_text(q_clean),
                "answer": a_clean,
                "answer_tokens": tokenize_text(a_clean)
            })
    return all_entries

# Updated function to handle all sheets including product-specific tabs
def load_and_process_excel_all_sheets(file_path):
    xl = pd.ExcelFile(file_path)
    processed_data = {}

    for sheet in xl.sheet_names:
        df = xl.parse(sheet)
        df = df.fillna('')  # replace NaNs with empty strings

        sheet_data = []
        for idx, row in df.iterrows():
            row_data = {}
            for col, val in row.items():
                raw = str(val)
                cleaned = clean_text(raw)
                tokens = tokenize_text(cleaned)

                row_data[col if col else f"col_{idx}"] = {
                    "raw": raw,
                    "cleaned": cleaned,
                    "tokens": tokens
                }

            # Filter out completely empty rows
            if any(cell["cleaned"] for cell in row_data.values()):
                sheet_data.append(row_data)

        processed_data[sheet] = sheet_data

    return processed_data

# Save output to JSON
def save_preprocessed_json(data, out_path):
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

# Run it on your Excel
if __name__ == "__main__":

    # Execute preprocessing

    faq_path = '../data/funds_transfer_app_features_faq.json'
    faq_data = load_and_preprocess_faq(faq_path)
    with open("data_faq.json", "w") as f:
      json.dump(faq_data, f, indent=2)

    product_info_path = "../data/CC93C000.xlsx"
    output_dir = "preprocessed_data"
    os.makedirs(output_dir, exist_ok=True)

    all_sheets_cleaned = load_and_process_excel_all_sheets(product_info_path)
    save_preprocessed_json(all_sheets_cleaned, os.path.join(output_dir, "product_info_cleaned_all_sheets.json"))

    print("✅ All sheets cleaned and saved to `product_info_cleaned_all_sheets.json` and `faq_cleaned.json`.")


✅ All sheets cleaned and saved to `product_info_cleaned_all_sheets.json` and `faq_cleaned.json`.


In [41]:
df

,Unnamed: 0,Value Plus Current Account,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Main
0,NaN,I would like to inquire about opening a curren...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NUST Value Plus Current Account is specially d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,What are the free facilities being provided wi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,- Cheque Books (unlimited),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,- Debit Card (First is free)*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,- 2 Supplementary Debit Cards,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,- Duplicate Account Statements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,- On-line Fund Transfer Facility,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,- Free SMS alerts facility on Digital Transac...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,- Free E-statement & I-Net Banking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
question_keywords = ['what', 'how', 'when', 'why', 'can', 'does', 'is', 'do', 'are', 'who', 'should', 'would']
def is_question(text):
    if not isinstance(text, str):
        return False
    text = text.strip().lower()
    return any(text.startswith(q) for q in question_keywords) or text.endswith('?')


In [43]:
# Extract Q&A pairs
qa_pairs = []
i = 0
while i < len(df):
    text = str(df.at[i, main_col]) if pd.notna(df.at[i, main_col]) else ""

    if is_question(text):
        question = text.strip()
        i += 1
        answer_lines = []

        while i < len(df):
            content = str(df.at[i, main_col]) if pd.notna(df.at[i, main_col]) else ""
            if is_question(content):
                break
            if content.strip():
                answer_lines.append(content.strip())
            i += 1

        answer = "\n".join(answer_lines)
        qa_pairs.append({"question": question, "answer": answer})
    else:
        i += 1


In [44]:
# Output JSON format
faq_json = {
    "categories": [
        {
            "category": "Value Plus Current Account",
            "questions": qa_pairs
        }
    ]
}

# Save to file
with open("data_faq.json", "w", encoding="utf-8") as f:
    json.dump(faq_json, f, indent=2, ensure_ascii=False)

print("✅ All done! Check little_champs_faq.json")

✅ All done! Check little_champs_faq.json


In [45]:
df2 = xls.parse(xls.sheet_names[8])

In [46]:
df2

,Unnamed: 0,Value Plus Business Account,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Main
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,I would like to inquire about opening a curren...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NUST Value Plus Business Account is best suite...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Please tell me about Free Facilities being pr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Issuance of Cheque Books,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Duplicate Account Statement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,Issuance of Debit Card*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Internet Banking facility,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,SMS Alerts on Digital Transaction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,E-statement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
df2.dropna(how='all', inplace=True)
df2.reset_index(drop=True, inplace=True)

# Identify the main content column (not 'Unnamed')
main_col = next(col for col in df2.columns if "Unnamed" not in col)

In [48]:
df2

,Unnamed: 0,Value Plus Business Account,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Main
0,NaN,I would like to inquire about opening a curren...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NUST Value Plus Business Account is best suite...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Please tell me about Free Facilities being pr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Issuance of Cheque Books,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Duplicate Account Statement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Issuance of Debit Card*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,Internet Banking facility,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,SMS Alerts on Digital Transaction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,E-statement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,Inter Branch Online Cash Deposit/Withdrawal (O...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
question_keywords = ['what', 'how', 'when', 'why', 'can', 'does', 'is', 'do', 'are', 'who', 'should', 'would']
def is_question(text):
    if not isinstance(text, str):
        return False
    text = text.strip().lower()
    return any(text.startswith(q) for q in question_keywords) or text.endswith('?')


In [50]:
# Extract Q&A pairs
qa_pairs = []
i = 0
while i < len(df2):
    text = str(df2.at[i, main_col]) if pd.notna(df2.at[i, main_col]) else ""

    if is_question(text):
        question = text.strip()
        i += 1
        answer_lines = []

        while i < len(df2):
            content = str(df2.at[i, main_col]) if pd.notna(df2.at[i, main_col]) else ""
            if is_question(content):
                break
            if content.strip():
                answer_lines.append(content.strip())
            i += 1

        answer = "\n".join(answer_lines)
        qa_pairs.append({"question": question, "answer": answer})
    else:
        i += 1


In [51]:
faq_json = {
    "categories": [
        {
            "category": "Value Plus Business Account",
            "questions": qa_pairs
        }
    ]
}

with open("data_faq.json", "w", encoding="utf-8") as f:
    json.dump(faq_json, f, indent=2, ensure_ascii=False)

print("✅ Extracted and saved to value_plus_faq.json")

✅ Extracted and saved to value_plus_faq.json
